<a href="https://colab.research.google.com/github/candiceevemiller/personal-projects/blob/main/Wordle_Cheat_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/a-vanderheiden/WordleLetterDistribution/main/WordleAnswers.txt

--2022-02-01 17:22:22--  https://raw.githubusercontent.com/a-vanderheiden/WordleLetterDistribution/main/WordleAnswers.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18521 (18K) [text/plain]
Saving to: ‘WordleAnswers.txt.2’

WordleAnswers.txt.2 100%[===================>]  18.09K  --.-KB/s    in 0s      

2022-02-01 17:22:22 (75.4 MB/s) - ‘WordleAnswers.txt.2’ saved [18521/18521]



In [ ]:
wordle_answers = []
with open('WordleAnswers.txt', 'r') as f:
  for line in f:
    for word in line[1:-1].split(','):
      wordle_answers.append(word.strip('"'))
wordle_answers.sort()
wordle_answers[:25]

['aback',
 'abase',
 'abate',
 'abbey',
 'abbot',
 'abhor',
 'abide',
 'abled',
 'abode',
 'abort',
 'about',
 'above',
 'abuse',
 'abyss',
 'acorn',
 'acrid',
 'actor',
 'acute',
 'adage',
 'adapt',
 'adept',
 'admin',
 'admit',
 'adobe',
 'adopt']

In [ ]:
import re
def potential_answers(green: dict, yellow: dict, gray: str) -> list:
  """
  Takes 2 dicts, green, yellow, and a string gray
  example expected format:
  green = {1:'a', 2:'', 3:'', 4:'', 5:'e'}
  yellow = {1:'', 2:'ok', 3:'', 4: '', 5:''}
  gray = 'brlm'
  """
  search_sequence = ['.' for i in range(5)]  # create list equivalent to string '.....'
  # iterate over green dict and if the value isn't empty replace the character
  # at the matching index in search_sequence
  for key, val in green.items():
    if val:
      search_sequence[key-1] = val
  # create an empty string to store letters we know are in the word but don't know where
  # then we iterate over yellow dict and create a regex NOT term for each
  # non-empty index and append the relavent terms to the in_word variable
  # this way we know to look for words that have a letter but not at the Xth index
  in_word = ''
  for key, val in yellow.items():
    in_word += val
    if val and search_sequence[key-1] == '.':
      search_sequence[key-1] = '[^' + val + ']'
  # converts search_sequence list into a string then into a raw string and slices
  # off the single quotes introduced through the repr function
  search_sequence = repr(''.join(search_sequence))[1:-1]
  r = re.compile(search_sequence)
  # filters based on green known letters and which letters aren't at an index
  # but doesn't enforce the presence of yellow letters
  matches = list(filter(r.match, wordle_answers))
  # iterate through each word in matches, then iterates through every letter
  # we know must be in the word, if all letters are in the word we append the
  # word to our matches2 list
  matches2 = []
  for word in matches:
    flag = True
    for letter in in_word:
      if letter not in word:
        flag = False
    if flag:
      matches2.append(word)
  # iterate through every word in our matches2 list and then iterate through
  # every letter we know NOT to be in the word. If it contains none of the gray
  # letters we append our word to our final output list
  final_matches = []
  for word in matches2:
    flag = True
    for letter in gray:
      if letter in word:
        flag = False
    if flag:
      final_matches.append(word)
  print('Your potential matches are:\n')
  return final_matches

In [ ]:
green = {1:'', 2:'', 3:'', 4:'', 5:'e'}
yellow = {1:'s', 2:'et', 3:'', 4: '', 5:''}
gray = 'paryl'
potential_answers(green, yellow, gray)

Your potential matches are:



['these', 'those']

In [ ]:
def get_green() -> dict:
  """
  Gets and validates user input for green letters and formats for feeding into
  the function potential_answers()
  """
  green = {1:'', 2:'', 3:'', 4:'', 5:''}  # initialize green dict
  print("Let's get your green letters:")
  print("-----------------------------")
  for i in range(1,6):
    flag = 0  # helps us validate user, keeping us in the while loop if not y/n
    while flag != 'y' and flag != 'n':
      flag = input(f'Do you know what letter is in spot {i}? (y/n)\n')
      if flag != 'y' and flag !='n':
        print('Please only type y or n right now\n')
    if flag == 'y':
      letter = '0'
      while not letter.isalpha():
        letter = input(f'What letter do you have for spot {i}?\n')
        if not letter.isalpha():
          print('Please only type a letter right now')
      # store standardized user input in the correct index of green dict
      green[i] = letter.lower()
    else:
      continue
  return green

In [ ]:
def get_yellow() -> dict:
  """
  Gets and validates user input for yellow letters and formats for passing into
  function potential_answers()
  """
  yellow = {1:'', 2:'', 3:'', 4:'', 5:''}
  print("Let's get your yellow letters:")
  print("-----------------------------")
  for i in range(1,6):
    flag = 0 # again helps us validate user input
    while flag != 'y' and flag != 'n':
      flag = input(f'Do you know a yellow letter for spot {i}? (y/n)\n')
      if flag != 'y' and flag !='n':
        print('Please only type y or n right now\n')
    if flag == 'y':
      letter = '0'
      while not letter.isalpha():
        letter = input(f'What yellow letters do you know for spot {i}?\n(Type all that apply)\n')
        if not letter.isalpha():
          print('Please only type a letter right now')
      # stores standardized user input in correct index of yellow dict
      yellow[i] = letter.lower()
    else:
      continue
  return yellow

In [ ]:
def get_gray() -> str:
  """
  Gets and validates user input for gray letters and formats for passing into
  function potential_answers
  """
  gray = ''
  print(f"Let's get your gray letters:")
  print('-----------------------------')
  while not gray and not gray.isalpha():
    gray = input(f'What are your gray letters?\n(Type all that apply)\n')
    if not gray.isalpha():
      print('Please only type letters')
  return gray.lower()

In [ ]:
green = get_green()
yellow = get_yellow()
gray = get_gray()

potential_answers(green, yellow, gray)

Let's get your green letters:
-----------------------------
Do you know what letter is in spot 1? (y/n)
n
Do you know what letter is in spot 2? (y/n)
n
Do you know what letter is in spot 3? (y/n)
n
Do you know what letter is in spot 4? (y/n)
n
Do you know what letter is in spot 5? (y/n)
n
Let's get your yellow letters:
-----------------------------
Do you know a yellow letter for spot 1? (y/n)
n
Do you know a yellow letter for spot 2? (y/n)
y
What yellow letters do you know for spot 2?
(Type all that apply)
e
Do you know a yellow letter for spot 3? (y/n)
n
Do you know a yellow letter for spot 4? (y/n)
n
Do you know a yellow letter for spot 5? (y/n)
y
What yellow letters do you know for spot 5?
(Type all that apply)
s
Let's get your gray letters:
-----------------------------
What are your gray letters?
(Type all that apply)
par
Your potential matches are:



['bused',
 'chest',
 'chose',
 'close',
 'ensue',
 'exist',
 'flesh',
 'goose',
 'guest',
 'guise',
 'house',
 'islet',
 'issue',
 'loose',
 'louse',
 'moose',
 'mouse',
 'noise',
 'noose',
 'nosey',
 'obese',
 'onset',
 'quest',
 'scene',
 'scent',
 'scone',
 'sheen',
 'sheet',
 'sheik',
 'shelf',
 'shell',
 'shied',
 'shine',
 'shone',
 'shove',
 'siege',
 'sieve',
 'since',
 'sinew',
 'singe',
 'sleek',
 'sleet',
 'slice',
 'slide',
 'slime',
 'smell',
 'smelt',
 'smile',
 'smite',
 'smoke',
 'smote',
 'snide',
 'solve',
 'steed',
 'steel',
 'stein',
 'stoke',
 'stole',
 'stone',
 'stove',
 'style',
 'suite',
 'sweet',
 'swell',
 'swine',
 'these',
 'those',
 'unset',
 'whose']